In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

from art.attacks.evasion import ProjectedGradientDescent
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np
import pandas as pd

from sklearn import metrics
from warnings import simplefilter

import joblib

2024-02-26 20:48:11.239267: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 20:48:11.242049: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 20:48:11.273245: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 20:48:11.273289: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 20:48:11.274854: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test.reshape(-1), y_pred_classes, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [3]:
x_train = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/x_train.csv', low_memory=False)
y_train = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/y_train.csv', low_memory=False)
x_test = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/x_test.csv', low_memory=False)
y_test = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/y_test.csv', low_memory=False)

x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()

In [4]:
model = joblib.load("/home/jovyan/UNSW/dl/new_dl/dnn.joblib")

2024-02-26 20:48:16.823331: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://92ab165426bf4b19874efd329b494dea: INVALID_ARGUMENT: ram://92ab165426bf4b19874efd329b494dea is a directory.


In [5]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=2,
    input_shape=(56,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [6]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.01)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [7]:
calculate_performance_metrics(adv_test_examples, y_test, model)

5480/5480 [==============================] - 3s 559us/step
Accuracy: 0.9041182609885879
AUC: 0.9135551922342579

macro
Precision: 0.8828585491637047
Recall: 0.913555192234258
F1 Score: 0.8943625013609735

weighted
Precision: 0.9140091981886632
Recall: 0.9041182609885879
F1 Score: 0.9059593779786471

Mean FNR: 0.08644480776574211
Mean TNR: 0.913555192234258
Mean FPR: 0.08644480776574211
Mean TPR: 0.913555192234258


In [8]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.05)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [9]:
calculate_performance_metrics(adv_test_examples, y_test, model)

5480/5480 [==============================] - 3s 604us/step
Accuracy: 0.8894953262499928
AUC: 0.893292422628674

macro
Precision: 0.8674064006392516
Recall: 0.8932924226286739
F1 Score: 0.8775625640346358

weighted
Precision: 0.8977320533065243
Recall: 0.8894953262499928
F1 Score: 0.8913705088542878

Mean FNR: 0.1067075773713261
Mean TNR: 0.8932924226286739
Mean FPR: 0.1067075773713261
Mean TPR: 0.8932924226286739


In [6]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.1)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [7]:
adv_test_examples

array([[-0.1,  0.1, -0.1, ...,  0.9,  0.9,  0.1],
       [-0.1,  0.1, -0.1, ...,  0.9,  6.1,  0.1],
       [-0.1,  0.1, -0.1, ...,  1.9,  6.1,  0.1],
       ...,
       [-0.1, -0.1, -0.1, ...,  2.9, 11.9,  0.1],
       [-0.1, -0.1, -0.1, ..., 25.9, 29.9,  0.1],
       [-0.1, -0.1, -0.1, ..., 26.1, 29.9,  0.1]], dtype=float32)

In [8]:
calculate_performance_metrics(adv_test_examples, y_test, model)

5480/5480 [==============================] - 3s 566us/step
Accuracy: 0.8642930061993487
AUC: 0.8671960146165788

macro
Precision: 0.8404221006533991
Recall: 0.8671960146165789
F1 Score: 0.850350511497591

weighted
Precision: 0.8750225955888895
Recall: 0.8642930061993487
F1 Score: 0.8668514748179205

Mean FNR: 0.13280398538342109
Mean TNR: 0.8671960146165789
Mean FPR: 0.13280398538342109
Mean TPR: 0.8671960146165789


In [9]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.15)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [10]:
adv_test_examples

array([[-0.15,  0.15, -0.15, ...,  0.85,  0.85,  0.15],
       [-0.15,  0.15, -0.15, ...,  0.85,  6.15,  0.15],
       [-0.15,  0.15, -0.15, ...,  1.85,  6.15,  0.15],
       ...,
       [-0.15, -0.15, -0.15, ...,  2.85, 11.85,  0.15],
       [-0.15, -0.15, -0.15, ..., 25.85, 29.85,  0.15],
       [-0.15, -0.15, -0.15, ..., 25.85, 29.85,  0.15]], dtype=float32)

In [11]:
calculate_performance_metrics(adv_test_examples, y_test, model)

5480/5480 [==============================] - 3s 574us/step
Accuracy: 0.8173330823937356
AUC: 0.8270305107842233

macro
Precision: 0.7941605145632976
Recall: 0.8270305107842234
F1 Score: 0.8027486662085808

weighted
Precision: 0.8400069496389738
Recall: 0.8173330823937356
F1 Score: 0.8221243007902305

Mean FNR: 0.17296948921577665
Mean TNR: 0.8270305107842234
Mean FPR: 0.17296948921577665
Mean TPR: 0.8270305107842234


In [12]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.2)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [13]:
calculate_performance_metrics(adv_test_examples, y_test, model)

5480/5480 [==============================] - 3s 566us/step
Accuracy: 0.7409333812399838
AUC: 0.7670339465421415

macro
Precision: 0.7327323968179151
Recall: 0.7670339465421416
F1 Score: 0.7295875682401609

weighted
Precision: 0.7939099441360459
Recall: 0.7409333812399838
F1 Score: 0.7495968994914636

Mean FNR: 0.23296605345785848
Mean TNR: 0.7670339465421416
Mean FPR: 0.23296605345785848
Mean TPR: 0.7670339465421416


In [14]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.25)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [15]:
calculate_performance_metrics(adv_test_examples, y_test, model)

5480/5480 [==============================] - 3s 566us/step
Accuracy: 0.6894679510211531
AUC: 0.7269326937694744

macro
Precision: 0.6976919597334514
Recall: 0.7269326937694744
F1 Score: 0.6814251608500221

weighted
Precision: 0.766044792149891
Recall: 0.6894679510211531
F1 Score: 0.6997108167089455

Mean FNR: 0.2730673062305255
Mean TNR: 0.7269326937694744
Mean FPR: 0.2730673062305255
Mean TPR: 0.7269326937694744


In [6]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.3)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [7]:
calculate_performance_metrics(adv_test_examples, y_test, model)

5480/5480 [==============================] - 3s 562us/step
Accuracy: 0.6444356995796762
AUC: 0.6827620025209873

macro
Precision: 0.6599393496983328
Recall: 0.6827620025209873
F1 Score: 0.6372703312244625

weighted
Precision: 0.7298188560077205
Recall: 0.6444356995796762
F1 Score: 0.6556870491674743

Mean FNR: 0.3172379974790127
Mean TNR: 0.6827620025209873
Mean FPR: 0.3172379974790127
Mean TPR: 0.6827620025209873
